In [ ]:
import sys
import os

os.chdir("..")
os.chdir("./src")
# sys.path.append("./src")

In [ ]:
from IPython import display
from IPython.display import clear_output
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, jit
from time import time

# from helpers import *
from LDMIBSS import *

# np.random.seed(13)
%load_ext autoreload
%autoreload 2

In [ ]:
N = 500000
NumberofSources = 5
NumberofMixtures = 10
S = generate_correlated_copula_sources(
    rho=0.0,
    df=4,
    n_sources=NumberofSources,
    size_sources=N,
    decreasing_correlation=True,
)
# S = 2 * S - 1
sparse_components = [0, 1, 3]
# nn_components = [4]
S[sparse_components] = 2 * S[sparse_components] - 1
S[sparse_components] = ProjectRowstoL1NormBall(S[sparse_components].T).T

print("The following is the correlation matrix of sources")
display_matrix(np.corrcoef(S))

plt.scatter(S[0, :], S[2, :])
plt.show()

plt.scatter(S[0, :], S[1, :])
plt.show()

plt.scatter(S[0, :], S[4, :])
plt.show()

In [ ]:
# Generate Mxr random mixing from i.i.d N(0,1)
A = np.random.randn(NumberofMixtures, NumberofSources)
X = np.dot(A, S)

print("The following is the mixture matrix A")
display_matrix(A)

SNR = 30  # dB
NoiseAmp = 10 ** (-SNR / 20)  # * np.sqrt(NumberofSources)
NoisePart = NoiseAmp * np.random.randn(X.shape[0], X.shape[1])
X = X + NoisePart
SNRinp = 10 * np.log10(np.sum(np.var(X, axis=1)) / np.sum(np.var(NoisePart, axis=1)))
print("Input SNR is : {}".format(SNRinp))

In [ ]:
subplot_1D_signals(
    S.T[0:100], title="Original Signals", figsize=(15.2, 9), colorcode=None
)
subplot_1D_signals(
    X.T[0:100], title="Mixture Signals", figsize=(15, 18), colorcode=None
)

In [ ]:
lambday = 1 - 1e-1 / 10
lambdae = 1 - 1e-7 / 10
beta = 10
s_dim = S.shape[0]
x_dim = X.shape[0]

# Inverse output covariance
By = 5 * np.eye(s_dim)
# Inverse error covariance
Be = 1 * np.eye(s_dim)

debug_iteration_point = 1000
model = OnlineLDMIBSS(
    s_dim=s_dim,
    x_dim=x_dim,
    muW=1e-3,
    lambday=lambday,
    lambdae=lambdae,
    beta=beta,
    By=By,
    Be=Be,
    neural_OUTPUT_COMP_TOL=1e-6,
    set_ground_truth=True,
    S=S,
    A=A,
)

In [ ]:
model.fit_batch_nnwsubsparse(
    X=X,
    sparse_components=sparse_components,
    n_epochs=1,
    neural_dynamic_iterations=500,
    plot_in_jupyter=True,
    neural_lr_start=0.2,
    neural_lr_stop=0.001,
    debug_iteration_point=debug_iteration_point,
    shuffle=True,
)